In [0]:
from google.colab import auth
auth.authenticate_user()

In [2]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=2yieHNFmol7XWxrk8YPKBTmUgyHFvGYAlNo-uMas-zg&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/zQEyDdjnI9dEgVA8odIAnqSW7--P0yl_wi_ndmZqD2IIjv5kyyd9jYQ

Successfully saved authorization token.


In [3]:
import tensorflow as tf

print(tf.__version__)

2.2.0-rc3


In [4]:
import folium
print(folium.__version__)

0.8.3


In [0]:
BUCKET = 'andrewrenningerfinal'

In [0]:
FOLDER = 'final'
TRAINING_BASE = 'training'
EVAL_BASE = 'testing'

# Specify inputs (Landsat bands) to the model and the response variable.
opticalBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
thermalBands = ['B10', 'B11']
BANDS = opticalBands + thermalBands
RESPONSE = 'impervious'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 16000
EVAL_SIZE = 8000

# Specify model training parameters.
BATCH_SIZE = 16
EPOCHS = 10
BUFFER_SIZE = 2000
OPTIMIZER = 'SGD'
LOSS = 'MeanSquaredError'
METRICS = ['accuracy']

In [7]:
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
srtm = ee.Image('USGS/SRTMGL1_003')

def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  mask2 = image.mask().reduce('min')
  mask3 = image.select(opticalBands).gt(0).And(
          image.select(opticalBands).lt(10000)).reduce('min')
  mask = mask1.And(mask2).And(mask3)
  return image.select(opticalBands).divide(10000).addBands(
          image.select(thermalBands).divide(10).clamp(273.15, 373.15)
            .subtract(273.15).divide(100)).updateMask(mask)

image = l8sr.filterDate('2015-01-01', '2017-12-31').map(maskL8sr).median()

mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[40, -75.3])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)

mapid = image.getMapId({'bands': ['B10'], 'min': 0, 'max': 0.5})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='thermal',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [9]:
nlcd = ee.Image('USGS/NLCD/NLCD2016').select('impervious')
nlcd = nlcd.divide(100).float()

mapid = nlcd.getMapId({'min': 0, 'max': 1})
map = folium.Map(location=[40, -75.3])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='nlcd impervious',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [0]:
light = image.select(BANDS)
slope = ee.Terrain.slope(srtm) 

composite = nlcd.addBands([slope, light])

In [51]:
mapid = composite.getMapId({'bands': ['slope'], 'min': 0, 'max': 90})
map = folium.Map(location=[40, -75.3])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)

map

composite = composite.float()

In [52]:
trainingPolys = ee.FeatureCollection('users/asrenninger/training')
evalPolys = ee.FeatureCollection('users/asrenninger/testing')

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[38., -100.], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [0]:
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())

for g in range(trainingPolys.size().getInfo()):
  desc = TRAINING_BASE + '_' + str(g)
  task = ee.batch.Export.image.toCloudStorage(
    image = composite,
    fileNamePrefix = FOLDER + '/' + desc,
    description = desc, 
    bucket = BUCKET, 
    region = ee.Feature(trainingPolysList.get(g)).geometry(), 
    scale = 30, 
  )
  task.start()

In [0]:
for g in range(evalPolys.size().getInfo()):
  desc = EVAL_BASE + '_' + str(g)
  task = ee.batch.Export.image.toCloudStorage(
    image = composite,
    fileNamePrefix = FOLDER + '/' + desc,
    description = desc, 
    bucket = BUCKET, 
    region = ee.Feature(evalPolysList.get(g)).geometry(), 
    scale = 30, 
  )
  task.start()